In [ ]:
import copy
%matplotlib inline

import refnx
from refnx.reflect import SLD
from hogben.utils import Fisher
from hogben.optimise import Optimiser
from hogben.models.samples import Sample, MagneticLayerSLD

def simple_sample():
    """Define a bilayer sample, and return the associated refnx model"""
    # Define sample

    layer1_thick = refnx.analysis.Parameter(50, 'Layer1 thickness', (30, 150))
    layer2_thick = refnx.analysis.Parameter(40, 'Layer2 thickness', (30, 150))
    layer1_rough = refnx.analysis.Parameter(6, 'Layer1 Roughness', (2, 10))
    layer2_rough = refnx.analysis.Parameter(7, 'Layer2 roughness', (2, 10))

    ul_thick = refnx.analysis.Parameter(50, 'Underlayer thickness', (0, 350))
    ul_nsld = refnx.analysis.Parameter(2, 'Underlayer Nuclear SLD', (1.5, 6))
    ul_msld = refnx.analysis.Parameter(2, 'Underlayer Magnetic SLD', (0, 6))    

    air = refnx.reflect.SLD(0, name='Air')    
    layer1 = refnx.reflect.SLD(3, name="Layer 1")(thick=layer1_thick, rough=layer1_rough)
    layer2 = refnx.reflect.SLD(-0.5, name="Layer 2")(thick=layer2_thick, rough=layer2_rough)

    underlayer = MagneticLayerSLD(SLDn=ul_nsld, SLDm=ul_msld, thick=ul_thick, rough=5, underlayer=True)    
    substrate = SLD(2.074, name='Substrate')(rough=3)




    params = [
        layer1_thick,
        layer2_thick,
        layer1_rough,
        layer2_rough,
    ]

    for param in params:
        param.vary = True

    optimize = [
        ul_msld,
        ul_thick,
        ul_nsld,
    ]

    for params in optimize:
        params.optimize = True

    sample = substrate | underlayer | layer1 | layer2 | layer1 | air
    return sample

sample = Sample(simple_sample())

angle_times = [(0.7, 100, 100),
               (2.3, 100, 400),
               ]


In [ ]:
optimiser = Optimiser(sample)
res, val = optimiser.optimise_parameters(angle_times, verbose=False)


print("The parameters with the highest information could be found at:")
for param, value in zip(sample.get_optimization_parameters(), res):
    print(f"{param.name}: {value}")
    param.value = value
print("----------------------")

In [ ]:
angle_times2 = [(0.7, 100, 200),
               (2.3, 100, 800),
               ]
sample_no_ul = copy.deepcopy(sample)._remove_underlayers()
fisher = Fisher.from_sample(sample, angle_times)
fisher_no_ul = Fisher.from_sample(sample_no_ul, angle_times2)
ratio = fisher.min_eigenval / fisher_no_ul.min_eigenval
print(f"Fisher with underlayer {fisher.min_eigenval:.4f}")
print(f"Fisher without underlayer {fisher_no_ul.min_eigenval:.4f}")
print(f"Improvement of {(ratio * 100 - 100):.2f}% with underlayer")